In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [ ]:
path_clean_data = "./clean_data.txt" # Chemin d'accès de clean_data.txt
path_clean_meta = "./clean_meta.txt" # Chemin d'accès de clean_meta.txt
path_clean_tax = "clean_tax.txt" # Chemin d'accès de clean_tax.txt

clean_data_df = pd.read_csv(path_clean_data,delim_whitespace=True) #Dataframe représentant clean_data
clean_meta_df = pd.read_csv(path_clean_meta,sep="\t").transpose() #Dataframe représentant clean_meta
clean_tax_df  = pd.read_csv(path_clean_tax,sep="\t") #Dataframe représentant clean_tax
clean_tax_df  = clean_tax_df.iloc[:,1:]

In [108]:
print(f"Pour les données clean_data_df : {np.shape(clean_data_df)}")
print(f"Pour les données clean_meta_df : {np.shape(clean_meta_df)}")
print(f"Pour les données clean_tax_df : {np.shape(clean_tax_df)}")


Pour les données clean_data_df : (1579, 1095)
Pour les données clean_meta_df : (36, 1095)
Pour les données clean_tax_df : (1579, 7)


# PHASE DE PRE-PROCESSING

Première des choses qu'on veut faire c'est joindre les données issues de `clean_data_df` et ceux issues de `clean_tax_df`.
Mais avant il faut vérifier que les individus issues de ces 2 bases de données sont les mêmes.
Pour ce faire, il faut : 
- Récupérer la liste d'index des individus de `clean_data_df` et celle de `clean_tax_df`.
- Faire une comparaison entre les deux et juste récupérer les similarités.
Par exemple : 
- Si index_1 = [0,1,40,26,100,157] et index_2 = [0,1,45,157,26], les indexes en commun sont : **0,1,26**

In [129]:
clean_data_indexes_list = list(clean_data_df.index)
clean_tax_indexes_list = list(clean_tax_df.index)
mask_list = [x for x in clean_tax_indexes_list if x in clean_data_indexes_list]
num_mismatches = len(clean_data_indexes_list) - len(mask_list)
print(f"Il y'a {len(mask_list)} ressemblances et {num_mismatches} différences au niveau des indexes dans les deux listes")

Il y'a 1175 ressemblances et 404 différences au niveau des indexes dans les deux listes


In [131]:
#Pour récupérer un tableau représentant les familles en lignes et leurs nombres dans chaque individu en colones
concat_df=pd.concat([clean_data_df,clean_tax_df],axis=1,verify_integrity=True,join="inner")
concat_df.shape

(1175, 1102)

In [ ]:
#Pour récupérer un tableau représentant les familles en lignes et leurs nombres dans chaque individu en colones
concat_df=pd.concat([clean_data_df,clean_tax_df],axis=1,verify_integrity=True,join="inner")
sum_family_per_person_df = concat_df.groupby("family").sum().astype(int)

#Pour récupérer un tableau représentant les familles en lignes et leur nombre totale dans toute la population
sum_family_for_all_person_df = pd.DataFrame(sum_family_per_person_df.sum(axis=1).astype(int))

# 1ère phase : Enlever ceux qui sont nulles puisqu'ils n'influencent pas et les nan values également
cond = sum_family_for_all_person_df[0] > 0 
sum_family_for_all_person_df_filtered_v1 = sum_family_for_all_person_df.loc[cond].dropna()
print(f"Il y'a {len(sum_family_for_all_person_df_filtered_v1)} familles qui sont nulles sur tout le df : {list(sum_family_for_all_person_df_filtered_v1.index)}")

# Faire une transformation sur les données pour prendre la racine afin de réduire la dispersion
sum_family_for_all_person_df_filtered_v2 = sum_family_for_all_person_df_filtered_v1.applymap(lambda x : np.sqrt(x))

# Récupérer le 1er Quantile (Q1) cad 25%
Q1 = sum_family_for_all_person_df_filtered_v2.quantile(0.25)[0]

# Récupérer le 3ème Quantile (Q3) cad 75%
Q3 = sum_family_for_all_person_df_filtered_v2.quantile(0.75)[0]

# Calculer l'IQR (Interquartile Range)
IQR = Q3 - Q1 

# Récupérer la moyenne 
mean = sum_family_for_all_person_df_filtered_v2.mean()[0]

# Récupérer la médiane 
med = sum_family_for_all_person_df_filtered_v2.median()[0]

# Récupérer l'écart type 
ecart_type = sum_family_for_all_person_df_filtered_v2.std()[0]

# Définir les bornes inférieure et supérieure
lower_bound = Q1 
upper_bound = Q3 

print(f"\n---------->INFORMATIONS SUR LA DISPERTION DES DONNÉES<---------- \n")
print(f"    Moyenne = {mean}")
print(f"    Ecart-type = {ecart_type}")
print(f"    Mediane = {med}")
print(f"    Q1 = {Q1}")
print(f"    Q3 = {Q3}")
print(f"    IQR = {IQR}")
print(f"    La borne inférieure est : {lower_bound}")
print(f"    La borne supérieure est : {upper_bound}")

#Filtrer les données pour ne garder que celles qui sont dans les bornes
sum_family_for_all_person_df_filtered_v2 = sum_family_for_all_person_df_filtered_v2[(sum_family_for_all_person_df_filtered_v2[0] >= lower_bound) & (sum_family_for_all_person_df_filtered_v2[0] <= upper_bound)]
print(f"\n Après filtrage final , il reste {len(sum_family_for_all_person_df_filtered_v2)} familles")

Il y'a 88 familles qui sont nulles sur tout le df : ['0319-6G20', 'Acetobacteraceae', 'Acidaminococcaceae', 'Actinomycetaceae', 'Aerococcaceae', 'Aeromonadaceae', 'Alcaligenaceae', 'Alicyclobacillaceae', 'Anaeroplasmataceae', 'Bacillaceae', 'Bacteroidaceae', 'Bdellovibrionaceae', 'Beijerinckiaceae', 'Bifidobacteriaceae', 'Bradyrhizobiaceae', 'Brevibacteriaceae', 'Brucellaceae', 'Burkholderiaceae', 'Campylobacteraceae', 'Carnobacteriaceae', 'Caulobacteraceae', 'Chitinophagaceae', 'Christensenellaceae', 'Clostridiaceae_1', 'Comamonadaceae', 'Coriobacteriaceae', 'Corynebacteriaceae', 'Deferribacteraceae', 'Defluviitaleaceae', 'Desulfovibrionaceae', 'Elusimicrobiaceae', 'Enterobacteriaceae', 'Enterococcaceae', 'Erysipelotrichaceae', 'Eubacteriaceae', 'Family_XI', 'Family_XIII', 'Flavobacteriaceae', 'Fusobacteriaceae', 'GR-WP33-58', 'Lachnospiraceae', 'Lactobacillaceae', 'Leptotrichiaceae', 'Leuconostocaceae', 'Methanobacteriaceae', 'Methylobacteriaceae', 'Methylophilaceae', 'Microbacteriac

: 

In [ ]:
# Il faut ensuite enlever les familles qui ont été filtré dans le df qu'on avait initialement
cond_1 = concat_df["family"].isin(list(sum_family_for_all_person_df_filtered_v2.index))
concat_df_filtered = concat_df[cond_1]
print(f"Avant filtrage de clean_data et clean_tax on a : {concat_df.shape[0]}")
print(f"Après filtrage de clean_data et clean_tax on a : {concat_df_filtered.shape[0]}")

Avant filtrage de clean_data et clean_tax on a : 1175
Après filtrage de clean_data et clean_tax on a : 251


In [128]:
concat_df_filtered

,DYEY1,DYEY10,DYEY2,DYEY3,DYEY4,DYEY5,DYEY6,DYEY7,DYEY8,DYEY9,...,R141,R174,R51,domain,phylum,order,class,family,genus,quality
16,0,5,1,26,2,0,55,4,49,0,...,352,1,83,Bacteria,Firmicutes,Bacilli,Lactobacillales,Enterococcaceae,Enterococcus,|100
83,0,0,1,0,0,0,0,0,2,0,...,1431,0,37,Archaea,Euryarchaeota,Methanobacteria,Methanobacteriales,Methanobacteriaceae,Methanobrevibacter,|100
89,8,0,0,5,0,0,0,0,0,0,...,0,1,0,Bacteria,Fusobacteria,Fusobacteriia,Fusobacteriales,Fusobacteriaceae,Fusobacterium,|99
106,49,38,365,155,3,0,45,83,22,0,...,277,0,2,Bacteria,Proteobacteria,Deltaproteobacteria,Desulfovibrionales,Desulfovibrionaceae,Bilophila,|100
120,80,21,39,100,65,38,76,55,52,34,...,8,8,6,Bacteria,Actinobacteria,Actinobacteria,Actinomycetales,Actinomycetaceae,Actinomyces,|100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,0,0,17,0,12,1,7,7,18,0,...,1,0,0,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Oxalobacteraceae,Massilia,|77
1398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,Bacteria,Firmicutes,Bacilli,Lactobacillales,Enterococcaceae,Vagococcus,|96
1427,149,1,15,596,5262,12,329,16,151,7,...,29,0,1,Bacteria,Proteobacteria,Alphaproteobacteria,Caulobacterales,Caulobacteraceae,Brevundimonas,|100
1448,0,17,0,0,0,0,0,0,0,0,...,0,0,0,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Oxalobacteraceae,Actimicrobium,|71
